In [1]:
import pandas as pd

In [2]:
import pyodbc 
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-D0942MUC\SQLEXPRESS;"
                      "Database=college;"
                      "Trusted_Connection=yes;")



In [3]:
Students=pd.read_sql("SELECT * FROM Students$",cnxn)
Courses = pd.read_sql("SELECT * FROM Courses$",cnxn)
Classrooms = pd.read_sql("SELECT * FROM Classrooms$",cnxn)
Departments = pd.read_sql("SELECT * FROM Departments$",cnxn)
Teachers = pd.read_sql("SELECT * FROM Teachers$",cnxn)
class_courses = pd.merge(Classrooms,Courses,how='left',on='CourseId')
cls_crs_dep = pd.merge(Departments,class_courses,how = 'left',left_on = 'DepartmentId',right_on = 'DepartmentID')
cls_crs_dep_teach = pd.merge(cls_crs_dep,Teachers,how='right',on = 'TeacherId')
cls_crs_dep_teach_stud = pd.merge(cls_crs_dep_teach,Students,how='inner',on='StudentId')
data=cls_crs_dep_teach_stud.copy()
data.head()

,DepartmentId,DepartmentName,CourseId,StudentId,degree,CourseName,DepartmentID,TeacherId,FirstName_x,LastName_x,Gender_x,FirstName_y,LastName_y,Gender_y
0,1.0,English,1.0,202.0,92.031070,English Begginers,1.0,15.0,Conor,Mckittrick,M,Autumn,Major,F
1,2.0,Science,4.0,202.0,73.387328,Mathematics 101,2.0,2.0,Amanda,Dworkin,F,Autumn,Major,F
2,2.0,Science,8.0,202.0,83.563932,Chemistry,2.0,14.0,Jeffrey,Batzel,M,Autumn,Major,F
3,2.0,Science,18.0,202.0,66.109849,R,2.0,17.0,Tanner,Fitzsimons,M,Autumn,Major,F
4,3.0,Arts,20.0,202.0,63.988269,History of Arts,3.0,8.0,Caroline,Knuesel,F,Autumn,Major,F


#### a.	מנהל המכללה ביקש לדעת כמה סטודנטים יש לפי יחידה (מחלקה).

In [4]:
a= data.groupby(data.DepartmentName).StudentId.nunique()
a=a.reset_index()
a.columns = ['Department Name','Students']
a

,Department Name,Students
0,Arts,137
1,English,84
2,Science,200
3,Sport,81


#### b.	המורה באנגלית צריך להתארגן וביקש לדעת כמה סטודנטים יש לו לפי כל קורס שהוא מעביר וסה"כ התלמידים בכל הקורסים שלו.

In [5]:
b= data[data.DepartmentID==1].groupby(data.CourseName).StudentId.nunique()
total_sum=data[data.DepartmentID==1].groupby(data.DepartmentId).nunique().iloc[0,3]

b=b.reset_index()
b.columns=['Course Name','numStudents']
print(b)
print('-----------------------')
print('Total' + "                             " + str(total_sum))

##total_sum=total_sum.reset_index()
##total_sum.columns = ['Total', 'Students']


            Course Name  numStudents
0      Advanced English           24
1     English Begginers           32
2  Proffesional English           38
-----------------------
Total                             84


#### c.	המרכז למדעים רוצה להבין כמה כיתות קטנות (מתחת ל-22) וכמה גדולות צריך עבור קורסים ביחידה (מחלקה) שלו.

In [6]:
c= data[data.DepartmentName=='Science'].groupby(data.CourseName).StudentId.nunique()

c=c.reset_index()
c.columns =['NumClassRooms','Students']

c.loc[c['Students']<22, 'Classize'] = 'Small'
c.loc[c ['Students']>=22 ,'Classize'] = 'Big'
Classroomsize= c.groupby(c.Classize).agg('count').iloc[:,0:1]
print(Classroomsize)


          NumClassRooms
Classize               
Big                  10
Small                 6


#### d.	סטודנטית שהיא פעילה פמיניסטית טוענת שהמכללה מעדיפה לקבל יותר גברים מאשר נשים. תבדקו האם הטענה מוצדקת (מבחינת כמותית, לא סטטיסטית).

In [7]:
d=Students.groupby(['Gender']).agg(['count']).iloc[:,0:1]
d

,StudentId
,count
Gender,
F,165
M,115


#### e.	באיזה קורסים אחוז הגברים / הנשים הינה מעל 70%?

In [8]:
Total=data[['StudentId']].groupby(data.CourseName).agg('count')

crsbygndr=data.groupby(['CourseName','CourseId','Gender_y']).StudentId.nunique()
e=pd.merge(crsbygndr,Total,how='left',on='CourseName')
e['Gndr_Stud_percent']=(e['StudentId_x']/e['StudentId_y'])*100.0
e=e[e['Gndr_Stud_percent']>70]
print (e)


            StudentId_x  StudentId_y  Gndr_Stud_percent
CourseName                                             
Sculpture            17           24          70.833333
Tenis                13           18          72.222222


### f.	בכל אחד מהיחידות (מחלקות), כמה סטודנטים (מספר ואחוזים) עברו עם ציון מעל 80?

In [52]:
Total_students=data.groupby(data.DepartmentName).StudentId.nunique()
#print(Total_students)
#data['over80']=0
#data.loc[((data.DepartmentId==1) & (data.degree>80)),'over80'] = 1
#data.loc[((data.DepartmentId==2) & (data.degree>80)),'over80'] = 1
#data.loc[((data.DepartmentId==3) & (data.degree>80)),'over80'] = 1
#data.loc[((data.DepartmentId==4) & (data.degree>80)),'over80'] = 1
#data
Students_over80=data[data.degree>80].groupby(data.DepartmentName).StudentId.nunique()
f=pd.merge(Total_students,Students_over80,how='inner',on='DepartmentName')
f['percent']=(f['StudentId_y'] / f['StudentId_x'])*100.0
f.reset_index()
f.columns=['TotalStudents','Students over80','Percentover80']
f

,TotalStudents,Students over80,Percentover80
DepartmentName,,,
Arts,137,61,44.525547
English,84,40,47.619048
Science,200,122,61.000000
Sport,81,39,48.148148


### g.	בכל אחד מהיחידות (מחלקות), כמה סטודנטים (מספר ואחוזים) לא עברו (ציון מתחת ל-60) ?

In [53]:
Total_students=data.groupby(data.DepartmentName).StudentId.nunique()
Students_less60=data[data.degree<60].groupby(data.DepartmentName).StudentId.nunique()
f=pd.merge(Total_students,Students_less60,how='inner',on='DepartmentName')
f['percent']=(f['StudentId_y'] / f['StudentId_x'])*100.0
f.reset_index()
f.columns=['TotalStudents','Students less60','Percentless60']
f

,TotalStudents,Students less60,Percentless60
DepartmentName,,,
Arts,137,3,2.189781
English,84,2,2.380952
Science,200,7,3.500000
Sport,81,1,1.234568


### h.	תדרגו את המורים לפי ממוצע הציון של הסטודנטים מהגבוהה לנמוך.

In [65]:
data['Teacher']=data['FirstName_x']+data['LastName_x']
h=data[['Teacher','degree']].groupby(['Teacher']).mean().sort_values(by='degree',ascending= False)
h

,degree
Teacher,
DarickHess,81.432703
JordynBaron,80.729494
HeatherMetcalf,80.440242
ZacharyHealy,79.365867
KayleaMonzingo,79.365537
DanielBland,78.908281
KeeganPrice,78.814631
ConorMckittrick,78.699492
MaxwellRomkee,78.673509
